In [1]:
import pandas as pd

df = pd.read_csv("Data/Aviation_Data.csv")


/var/folders/ym/68nrz1n97wj0gz5413bhpqs80000gn/T/ipykernel_2858/2692933316.py:3: DtypeWarning: Columns (6,7,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Data/Aviation_Data.csv")


Make
Cessna             22227
Piper              12029
CESSNA              4922
Beech               4330
PIPER               2841
                   ...  
Leonard Walters        1
Maule Air Inc.         1
Motley Vans            1
Perlick                1
ROYSE RALPH L          1
Name: count, Length: 8237, dtype: int64

In [55]:
df["Make"]=df["Make"].map(lambda x:str(x))
df["Make"]=df["Make"].map(lambda x:x.lower())

In [76]:
df[df["Make"].str.contains("airbus")]["Make"].value_counts()

Make
airbus                            291
airbus industrie                  164
airbus helicopters                 13
airbus helicopters inc              3
airbus industries                   1
airbus/eurocopter                   1
airbus helicopter                   1
airbus helicopters (eurocopte       1
airbus helicopters deutschland      1
Name: count, dtype: int64

In [74]:
df["Make"]=df["Make"].map(lambda x: "boeing" if "boeing" in x else x)
df["Make"][df["Make"].str.contains("boeing")].value_counts()

Make
boeing    2833
Name: count, dtype: int64

In [81]:
df["Make"]=df["Make"].map(lambda x: "airbus" if "airbus" in x else x)

6737     airbus
11775    airbus
13699    airbus
15282    airbus
17123    airbus
18024    airbus
18297    airbus
19134    airbus
20367    airbus
21771    airbus
21901    airbus
21981    airbus
22921    airbus
22950    airbus
23477    airbus
26315    airbus
26334    airbus
28379    airbus
28951    airbus
29119    airbus
29555    airbus
29616    airbus
29755    airbus
34019    airbus
34253    airbus
34439    airbus
37169    airbus
37173    airbus
37389    airbus
37574    airbus
38971    airbus
39104    airbus
39251    airbus
39811    airbus
39876    airbus
40584    airbus
40934    airbus
41114    airbus
41233    airbus
41531    airbus
41760    airbus
42113    airbus
42548    airbus
42758    airbus
43160    airbus
43401    airbus
43402    airbus
43735    airbus
43872    airbus
44348    airbus
          ...  
86883    airbus
87004    airbus
87088    airbus
87122    airbus
87194    airbus
87202    airbus
87352    airbus
87375    airbus
87501    airbus
87672    airbus
87736    airbus
87791   

In [82]:
df["Make"].value_counts()

Make
cessna                            27149
piper                             14870
beech                              5372
boeing                             2833
bell                               2722
nan                                1522
mooney                             1334
robinson                           1230
grumman                            1172
bellanca                           1045
hughes                              932
schweizer                           773
air tractor                         691
aeronca                             636
mcdonnell douglas                   608
maule                               589
champion                            519
airbus                              476
stinson                             439
aero commander                      429
de havilland                        422
luscombe                            414
aerospatiale                        388
taylorcraft                         383
north american                     

In [98]:
df["Make"]=df["Make"].map(lambda x: "britten-norman" if "britten" in x else x)

In [100]:
df[df["Make"].str.contains("dornier")]["Make"].value_counts()

Make
dornier              32
fairchild dornier     3
dornier gmbh          2
Name: count, dtype: int64

In [41]:
pd.set_option("display.min_rows", 1000)

In [50]:
df["Make"].value_counts().to_csv("test.csv")